<a href="https://colab.research.google.com/github/cris-her/Crypto/blob/master/LSTM_Deep_Learning_Bitcoin_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM Deep Learning Bitcoin Trading Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime

%matplotlib inline
plt.style.use('fivethirtyeight')

from sklearn.metrics import mean_squared_error

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [ ]:
from datetime import datetime

In [ ]:
from binance.client import Client

client = Client(api_key, api_secret)

In [ ]:
candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE)

In [ ]:
len(candles)

In [ ]:
candles[499] #time-open-high-low-close-volume-close_time

In [ ]:
price = np.array([float(candles[i][4]) for i in range(500)])

In [ ]:
time = np.array([int(candles[i][0]) for i in range(500)])

t = np.array([datetime.fromtimestamp(time[i]/1000).strftime('%H:%M:%S') for i in range(500)])

In [ ]:
price.shape

In [ ]:
plt.figure(figsize=(8,5))
plt.xlabel('Time Step')
plt.ylabel('Bitcoin Price $')
plt.plot(price)

In [ ]:
timeframe = pd.DataFrame({'Time':t,'Price $BTC':price})

timeframe #minute by minute price

In [ ]:
price = price.reshape(500,1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
scaler.fit(price[:374])

In [ ]:
price = scaler.transform(price)

In [ ]:
df = pd.DataFrame(price.reshape(100,5),columns=['First','Second','Third','Fourth','Target'])

In [ ]:
df.head()

In [ ]:
#75% train, 25% test

x_train = df.iloc[:74,:4]
y_train = df.iloc[:74,-1]

x_test = df.iloc[75:99,:4]
y_test = df.iloc[75:99,-1]

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
model = Sequential()

model.add(LSTM(20, return_sequence=True, input_shape=(4,1)))
model.add(LSTM(40, return_sequence=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')

model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=5, epochs=100)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Normalized Price')
plt.plot(y_test, label='True')
plt.plot(y_pred, label='Prediction')
plt.legend()

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(y_test), label='True')
plt.plot(scaler.inverse_transform(y_pred), label='Prediction')
plt.legend()

In [ ]:
testScore = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test),scaler.inverse_transform(y_pred)))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('RSquared :', '{:.2%}'.format(r2_score(y_test,y_pred))

In [ ]:
model.save('Bitcoin_model.h5')

In [ ]:
#from keras.models import load_model

# load model
#model = load_model('Bitcoin_model.h5')

# summarize model.
#model.summary()

Second Model

In [ ]:
from sklearn.svm import SVR

In [ ]:
#75% train, 25% test

trainX = df.iloc[:74,:4]
trainY = df.iloc[:74,-1]

testX = df.iloc[75:99,:4]
testY = df.iloc[75:99,-1]

In [ ]:
svr_linear = SVR(kernel='linear', C=1e3, gamma=0.1)
svr_linear.fit(trainX,trainY)

In [ ]:
predY = svr_linear.predict(testX)

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(testY), label='True')
plt.plot(scaler.inverse_transform(predY), label='Prediction')
plt.legend()

In [ ]:
testScore = np.sqrt(mean_squared_error(scaler.inverse_transform(testY),scaler.inverse_transform(predY)))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
print('RSquared :', '{:.2%}'.format(r2_score(testY,predY))

Hyperparameter Tuning

In [ ]:
param_grid = {'C': [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4],
              'gamma': np.logspace(-2, 2, 50),
              'epsilon': [0.1,0.2,0.5,0.3]}

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
svm_model = SVR(kernel='linear')

In [ ]:
grid_search = RandomizedSearchCV(svm_model,param_grid,scoring='r2',n_jobs=-1)

grid_search.fit(trainX, trainY)

print(grid_search.best_estimator_)

In [ ]:
svm_model = SVR(C=10000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=100.0,
                kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

svm_model.fit(trainX,trainY)

pred = svm_model.predict(testX)

In [ ]:
testScore = np.sqrt(mean_squared_error(scaler.inverse_transform(testY),scaler.inverse_transform(pred)))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
print('RSquared :', '{:.2%}'.format(r2_score(testY,pred))

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(testY), label='True')
plt.plot(scaler.inverse_transform(pred), label='Prediction')
plt.legend()

Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
ridge = RidgeCV()

In [ ]:
ridge.fit(trainX,trainY)

Rpred = ridge.predict(testX)

In [ ]:
testScore = np.sqrt(mean_squared_error(scaler.inverse_transform(testY),scaler.inverse_transform(Rpred)))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
print('RSquared :', '{:.2%}'.format(r2_score(testY,Rpred))

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(testY), label='True')
plt.plot(scaler.inverse_transform(Rpred), label='Prediction')
plt.legend()

Hyperparameter Tuning

In [ ]:
normal_price = np.array([float(candles[i][4]) for i in range(500)])

In [ ]:
df = pd.DataFrame(price.reshape(100,5),columns=['First','Second','Third','Fourth','Target'])

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
#75% train, 25% test

x_train_r = df.iloc[:74,:4]
y_train_r = df.iloc[:74,-1]

x_test_r = df.iloc[75:99,:4]
y_test_r = df.iloc[75:99,-1]

In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5,population_size=50, verbosity=2)
tpot.fit(x_train_r, y_train_r)

In [ ]:
tpred = tpot.predict(x_test_r)

In [ ]:
testScore = np.sqrt(mean_squared_error(y_test_r, tpred))
print('Test Score: %.2f RMSE' % (testScore))

print('RSquared :', '{:.2%}'.format(r2_score(y_test_r, tpred))

In [ ]:
tpot.export('bitcoin.py')

In [ ]:
plt.figure(figure=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(np.array(y_test_r).reshape(24,), label='True')
plt.plot(tpred, label='Prediction')
plt.legend()

Trading bot

In [ ]:
check = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)

check(499)

In [ ]:
index = [496,495,498,499]

candles = scaler.transform(np.array([float(check[i][4]) for i in index]).reshape(1,-1))

model_feed = candles.reshape(1,4,1)

scaler.inverse_transform(model.predict(model_feed)[0])[0] #at 7:08 PM prediction today 6th April

In [ ]:
#trading bot
symbol = 'BTCUSDT' #symbol
quantity = '0.05' #quantity to trade

order = False
index = [496,497,498,499]

while True:
  price = client.get_recent_trades(symbol = symbol)
  candle = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
  candles = scaler.transform(np.array([float(candle[i][4] for i in index]).reshape(1,-1))
  model_feed = candles.reshape(1,4,1)

  if order == False and float(price[len(price)-1]['price']) < float(scaler.inverse_transform(model.predict(model_feed)[0])[0]):
    
    #client.order_market_buy(symbol = symbol, quantity = quantity)
    order = True
    buy_price = client.get_order_book(symbol = symbol)['asks'][0][0]\
    print('Buy @Market Price :', float(buy_price), ' Timestamp :', str(datetime.now()))
  
  elif order == True and float(price[len(price)-1]['price'])-float(buy_price) >= 10:
    
    #client.order_market_sell(symbol = symbol, quantity = quantity)                       #fires sell order to exchange if condition
    order=False                                                                           #sets order = False and closes open position 
    sell_price = client.get_order_book(symbol=symbol)['bids'][0][0]                       #gets the highest bid for the market order
    print('Sell @Market Price :', float(sell_price),' Timestamp :',str(datetime.now()))   #print sell price and datetime
  
  else:
    pass